In [5]:
pip install requests beautifulsoup4 Pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
from PIL import Image, ImageDraw, ImageFont

# Function to fetch text from a website
def get_text_from_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = ' '.join([p.get_text() for p in soup.find_all('p')])
    return text

def generate_image(text, output_image_path='output_image.png'):
    # Image settings
    image_width = 800
    image_height = 600
    background_color = 'white'
    text_color = 'black'
    font_size = 20

    image = Image.new('RGB', (image_width, image_height), background_color)
    draw = ImageDraw.Draw(image)

    font = ImageFont.load_default()

    lines = []
    words = text.split()
    current_line = words[0]
    for word in words[1:]:
        bbox = draw.textbbox((0, 0), current_line + ' ' + word, font=font)
        if bbox[2] <= image_width:
            current_line += ' ' + word
        else:
            lines.append(current_line)
            current_line = word

    lines.append(current_line)

    y = 50
    for line in lines:
        draw.text((50, y), line, fill=text_color, font=font)
        y += font_size + 5

    
    image.save(output_image_path)
    print(f'Image saved to {output_image_path}')

if __name__ == "__main__":
    
    website_url = 'https://lingua.com/fr/francais/lecture/ville/'
    web_text = get_text_from_website(website_url)
    generate_image(web_text)


Image saved to output_image.png


In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image, ImageDraw, ImageFont
import tkinter as tk
from tkinter import filedialog

class WebTextToImageApp:
    def __init__(self, master):
        self.master = master
        self.master.title("WebTextToImage App")

        self.customization_options = {
            'background_color': 'lightblue',
            'text_color': 'darkgreen',
            'font_size': 24,
            'font_path': 'OpenSans-SemiboldItalic.ttf'
        }

        self.url_label = tk.Label(master, text="Enter URL:")
        self.url_entry = tk.Entry(master, width=50)
        self.url_label.pack(pady=10)
        self.url_entry.pack(pady=10)

        self.load_button = tk.Button(master, text="Load from URL", command=self.load_from_url)
        self.load_button.pack(pady=10)

        self.save_button = tk.Button(master, text="Save as Image", command=self.save_as_image)
        self.save_button.pack(pady=10)

    def load_from_url(self):
        website_url = self.url_entry.get()
        web_text = get_text_from_website(website_url)
        self.generate_image(web_text)

    def save_as_image(self):
        website_url = self.url_entry.get()
        web_text = get_text_from_website(website_url)

        # Allow user to customize before generating the image
        self.customization_options['font_path'] = filedialog.askopenfilename(
            title="Select Custom Font File",
            filetypes=[("TrueType Font Files", "*.ttf"), ("All Files", "*.*")]
        )

        self.generate_image(web_text, customization=self.customization_options)

    def generate_image(self, text, customization=None):
        # Image settings
        image_width = 800
        image_height = 600

        # Create a blank image
        image = Image.new('RGB', (image_width, image_height), self.customization_options['background_color'])
        draw = ImageDraw.Draw(image)

        # Set font and size
        font_path = ImageFont.truetype(self.customization_options['font_path'], self.customization_options['font_size'])

        # Wrap text to fit within the image width
        lines = []
        words = text.split()
        current_line = words[0]
        max_text_width = 0

        for word in words[1:]:
            bbox = draw.textbbox((0, 0), current_line + ' ' + word, font=font_path)
            if bbox[2] <= image_width:
                current_line += ' ' + word
            else:
                lines.append(current_line)
                current_line = word

            max_text_width = max(max_text_width, bbox[2])

        lines.append(current_line)

        # Adjust image width based on the maximum text width
        image_width = max(image_width, int(max_text_width) + 100)  # Add some padding

        # Create a new image with adjusted width
        image = Image.new('RGB', (image_width, image_height), self.customization_options['background_color'])
        draw = ImageDraw.Draw(image)

        # Draw text on the image
        y = 50
        for line in lines:
            draw.text((50, y), line, fill=self.customization_options['text_color'], font=font_path)
            y += self.customization_options['font_size'] + 5

        # Save the image
        output_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
        if output_path:
            image.save(output_path)
            print(f'Image saved to {output_path}')

def get_text_from_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = ' '.join([p.get_text() for p in soup.find_all('p')])
    return text

if __name__ == "__main__":
    root = tk.Tk()
    app = WebTextToImageApp(root)
    root.mainloop()


Image saved to C:/Users/HP/Downloads/temp_image.png


# ----------------------------------------INTERFACE AVEC AFFICHE L'IMAGE----------------------------

In [3]:
import requests
from bs4 import BeautifulSoup
from PIL import Image, ImageTk, ImageDraw, ImageFont
import PySimpleGUI as sg
import io

def get_text_from_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        text = ' '.join([p.get_text() for p in soup.find_all('p')])
        return text
    except requests.RequestException as e:
        print(f"Error loading URL: {e}")
        return None

def generate_image(text, customization=None):
    # Image settings
    image_width = 800
    image_height = 600

    # Create a blank image
    image = Image.new('RGB', (image_width, image_height), customization['background_color'])
    draw = ImageDraw.Draw(image)

    # Set font and size
    font_path = customization['font_path']
    font_size = customization['font_size']
    font = ImageFont.truetype(font_path, font_size)

    # Wrap text to fit within the image width
    lines = []
    words = text.split()
    current_line = words[0]
    max_text_width = 0

    for word in words[1:]:
        bbox = draw.textbbox((0, 0), current_line + ' ' + word, font=font)
        if bbox[2] <= image_width:
            current_line += ' ' + word
        else:
            lines.append(current_line)
            current_line = word

        max_text_width = max(max_text_width, bbox[2])

    lines.append(current_line)

    image_width = max(image_width, int(max_text_width) + 100)  # Add some padding

    image = Image.new('RGB', (image_width, image_height), customization['background_color'])
    draw = ImageDraw.Draw(image)

    y = 50
    for line in lines:
        draw.text((50, y), line, fill=customization['text_color'], font=font)
        y += font_size + 5

    return image

def main():
    sg.theme('LightBlue2')

    layout = [
        [sg.Text("Enter URL:"), sg.InputText(key='-URL-', size=(30, 1)), sg.Button('Load from URL')],
        [sg.Button('Select Font'), sg.Text('Font Path:'), sg.InputText(key='-FONT-', size=(40, 1))],
        [sg.Canvas(size=(800, 600), key='-CANVAS-', background_color='white')],
        [sg.Button('Save as Image')]
    ]

    window = sg.Window('WebTextToImage', layout, resizable=True)

    customization_options = {
        'background_color': 'lightblue',
        'text_color': 'darkgreen',
        'font_size': 24,
        'font_path': 'OpenSans-SemiboldItalic.ttf'
    }

    while True:
        event, values = window.read()

        if event == sg.WINDOW_CLOSED:
            break

        if event == 'Load from URL':
            url = values['-URL-']
            web_text = get_text_from_website(url)
            if web_text:
                img = generate_image(web_text, customization_options)
                img_tk = ImageTk.PhotoImage(img)
                window['-CANVAS-'].TKCanvas.create_image(0, 0, anchor='nw', image=img_tk)

        if event == 'Select Font':
            font_path = sg.popup_get_file("Select Custom Font File", file_types=(("TrueType Font Files", "*.ttf"), ("All Files", "*.*")))
            if font_path:
                customization_options['font_path'] = font_path
                window['-FONT-'].update(font_path)

        if event == 'Save as Image':
            web_text = get_text_from_website(values['-URL-'])
            if web_text:
                img = generate_image(web_text, customization_options)
                file_path = sg.popup_get_file("Save Image As", save_as=True, file_types=(("PNG files", "*.png"), ("All Files", "*.*")))
                if file_path:
                    img.save(file_path)

    window.close()

if __name__ == '__main__':
    main()
